# Misc Wrappers

## Common Wrappers

### class gymnasium.wrappers.TimeLimit(env: Env, max_episode_steps: int)

`gymnasium.wrappers.TimeLimit`是一个常用的环境包装器，用于限制环境中的最大步数。如果环境中的步数超过了设定的最大步数，环境将被截断，发出截断信号（`truncated`）。这与环境自身作为马尔可夫决策过程（MDP）的一部分产生的终止信号（`terminated`）不同。`TimeLimit`包装器是处理环境时间限制的主要方式，尤其是在环境自身没有定义截断逻辑时。

### 参数
- `env`: 要应用包装器的环境。
- `max_episode_steps`: 最大步数。如果为`None`，则使用`env.spec.max_episode_steps`。

### 使用场景
- 控制实验的最大步数，以便于比较不同策略或配置。
- 防止某些环境中的无限循环，通过设置步数上限来确保实验能够在合理的时间内完成。

### 示例

#### 示例1：使用`TimeLimit`包装器
```python
from gymnasium.wrappers import TimeLimit
from gymnasium.envs.classic_control import CartPoleEnv

# 创建一个CartPole环境实例，并应用TimeLimit包装器，设置最大步数为10步
env = TimeLimit(CartPoleEnv(), max_episode_steps=10)

# 输出环境信息，验证TimeLimit包装器已经应用
print(env)  # 输出：<TimeLimit<CartPoleEnv instance>>
```

#### 示例2：通过`gym.make`创建环境并设置最大步数
```python
import gymnasium as gym

# 使用gym.make创建环境，并通过参数设置最大步数为3步
env = gym.make("CartPole-v1", max_episode_steps=3)

# 重置环境并进行三次步进操作，观察截断信号
_ = env.reset(seed=123)
_ = env.action_space.seed(123)
for _ in range(3):
    _, _, terminated, truncated, _ = env.step(env.action_space.sample())
    print(f"Terminated: {terminated}, Truncated: {truncated}")
```

### 更改日志
- `v0.10.6` - 初始添加。
- `v0.25.0` - 随着步骤API更新，终止和截断信号被分开返回。

使用`TimeLimit`包装器可以有效地管理实验的步数限制，避免无限循环并控制实验的运行时间。通过分开处理`terminated`和`truncated`信号，它还允许更精细地控制实验结束的逻辑。

### class gymnasium.wrappers.RecordVideo(env: gym.Env[ObsType, ActType], video_folder: str, episode_trigger: Callable[[int], bool] | None = None, step_trigger: Callable[[int], bool] | None = None, video_length: int = 0, name_prefix: str = 'rl-video', fps: int | None = None, disable_logger: bool = True)

`gymnasium.wrappers.RecordVideo`类是一个包装器，用于记录环境剧集的视频，利用环境的渲染函数。通常，你可能只希望间歇性地记录剧集，例如每百个剧集或每千个环境步骤记录一次。为此，你可以指定`episode_trigger`或`step_trigger`。这些应该是返回布尔值的函数，指示是否应在当前剧集或步骤开始录制。

### 参数解读
- **env**：将被包装的环境。
- **video_folder** (`str`)：存储录制视频的文件夹。
- **episode_trigger**：一个接受整数并返回布尔值的函数，如果在此剧集应开始录制，则返回True。
- **step_trigger**：一个接受整数并返回布尔值的函数，如果在此步骤应开始录制，则返回True。
- **video_length** (`int`)：录制剧集的长度。如果为0，整个剧集被录制。否则，将捕获指定长度的片段。
- **name_prefix** (`str`)：将被添加到录制文件名前的前缀。
- **fps** (`int`)：视频中的帧率。如果提供了自定义视频fps，将使用此值；如果为None，则使用环境元数据中的`render_fps`键（如果存在）；否则使用默认值30。
- **disable_logger** (`bool`)：是否禁用moviepy日志记录器，默认禁用。

### 代码示例

#### 示例1：每10个剧集保存一次视频
```python
import os
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")
trigger = lambda t: t % 10 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./save_videos1", episode_trigger=trigger, disable_logger=True)

for i in range(50):
    termination, truncation = False, False
    _ = env.reset(seed=123)
    while not (termination or truncation):
        _, _, termination, truncation, _ = env.step(env.action_space.sample())

env.close()
print(len(os.listdir("./save_videos1")))  # 应该保存5个视频
```

#### 示例2：每200个步骤开始录制，并确保每个视频是100帧长
```python
import os
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")
trigger = lambda t: t % 200 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./save_videos2", step_trigger=trigger, video_length=100, disable_logger=True)

for i in range(5):
    termination, truncation = False, False
    _ = env.reset(seed=123)
    while not (termination or truncation):
        _, _, termination, truncation, _ = env.step(env.action_space.sample())

env.close()
print(len(os.listdir("./save_videos2")))  # 应该保存2个视频
```

#### 示例3：运行3个剧集，记录所有内容，但每1000帧划分一次
```python
import os
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder="./save_videos3", video_length=1000, disable_logger=True)

for i in range(3):
    termination, truncation = False, False
    _ = env.reset(seed=123)
    while not (termination or truncation):
        _, _, termination, truncation, _ = env.step(env.action_space.sample())

env.close()
print(len(os.listdir("./save_videos3")))  # 应该保存2个视频
```

### 更改日志
- `v0.25.0`：初始添加，替代了`wrappers.monitoring.VideoRecorder`。

这些示例展示了如何使用`RecordVideo`包装器以不同的触发条件和视频长度记录环境的视频。通过调整`episode_trigger`、`step_trigger`和`video_length`参数，可以灵活地控制视频录制的时机和持续时间。

### class gymnasium.wrappers.RecordEpisodeStatistics(env: Env[ObsType, ActType], buffer_length: int = 100, stats_key: str = 'episode')

`gymnasium.wrappers.RecordEpisodeStatistics`是一个用于跟踪累积奖励和剧集长度的包装器。在剧集结束时，该剧集的统计信息将通过`info`字典中的键`episode`添加。如果使用向量化环境，还会使用键`_episode`，这表明相应索引的环境是否具有剧集统计信息。存在一个向量版本的包装器，即`gymnasium.wrappers.vector.RecordEpisodeStatistics`。

### 参数解读
- **env** (`Env`): 要应用此包装器的环境。
- **buffer_length** (`int`): `return_queue`、`length_queue`和`time_queue`缓冲区的大小。
- **stats_key** (`str`): 用于剧集统计信息的`info`键。

### 功能
在剧集完成后，`info`将包含以下形式的统计信息：
```python
info = {
    "episode": {
        "r": "<cumulative reward>",
        "l": "<episode length>",
        "t": "<elapsed time since beginning of episode>"
    }
}
```
对于向量化环境，输出将采用以下形式：
```python
infos = {
    "final_observation": "<array of length num-envs>",
    "_final_observation": "<boolean array of length num-envs>",
    "final_info": "<array of length num-envs>",
    "_final_info": "<boolean array of length num-envs>",
    "episode": {
        "r": "<array of cumulative reward>",
        "l": "<array of episode length>",
        "t": "<array of elapsed time since beginning of episode>"
    },
    "_episode": "<boolean array of length num-envs>"
}
```
此外，最近的奖励和剧集长度存储在可以通过`wrapped_env.return_queue`和`wrapped_env.length_queue`分别访问的缓冲区中。

### 更改日志
- `v0.15.4` - 初始添加。
- `v1.0.0` - 移除了向量化环境支持（参见`gymnasium.wrappers.vector.RecordEpisodeStatistics`）并添加了`time_queue`属性。

### 使用示例
```python
import gymnasium as gym

env = gym.make("CartPole-v1")
env = gym.wrappers.RecordEpisodeStatistics(env)

observation = env.reset()
done = False
while not done:
    observation, reward, done, truncated, info = env.step(env.action_space.sample())
    if done or truncated:
        print(info['episode'])  # 打印剧集统计信息
```
这个示例演示了如何使用`RecordEpisodeStatistics`包装器来跟踪和打印累积奖励、剧集长度以及自剧集开始以来的经过时间。这对于评估和比较不同策略的性能非常有用。

### class gymnasium.wrappers.AtariPreprocessing(env: Env, noop_max: int = 30, frame_skip: int = 4, screen_size: int = 84, terminal_on_life_loss: bool = False, grayscale_obs: bool = True, grayscale_newaxis: bool = False, scale_obs: bool = False)

`gymnasium.wrappers.AtariPreprocessing`是一个专门为Atari环境设计的预处理包装器，实现了Atari环境常见的预处理技术（不包括帧堆叠）。如果需要帧堆叠，可以使用`gymnasium.wrappers.FrameStackObservation`。该类遵循了Machado等人在2018年的文章“Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents”中提出的指导原则。

### 预处理步骤
- **Noop Reset**：通过在重置时执行随机数量的无操作(no-ops)来获取初始状态，默认最大为30个no-ops。
- **帧跳过**：每个步骤之间跳过的帧数，默认为4。
- **最大池化**：对帧跳过期间最近的两次观察进行池化。
- **生命损失时发出终止信号**：当代理在环境中失去生命时，环境会终止。默认关闭。Machado等人（2018）不推荐使用。
- **调整为正方形图像**：将Atari环境原始观察形状从210x180调整为默认的84x84。
- **灰度观察**：将观察结果转换为灰度，默认启用。
- **灰度新轴**：如果启用`grayscale_obs`，并且此选项为True，则在灰度观察中添加一个通道轴，使图像变为三维，默认不启用。
- **观察范围缩放**：如果为True，则观察结果在[0,1)范围内归一化，否则为[0,255)。默认不缩放。这也限制了`FrameStack`包装器的内存优化好处。

### 示例代码
```python
import gymnasium as gym

# 创建一个Atari环境并应用预处理
env = gym.make("ALE/Adventure-v5")
env = gym.wrappers.AtariPreprocessing(env, noop_max=10, frame_skip=0, screen_size=84, 
                                      terminal_on_life_loss=True, grayscale_obs=False, 
                                      grayscale_newaxis=False)
```
这个示例创建了一个Atari环境，并对其应用了预处理，包括Noop Reset、调整屏幕大小、生命损失时终止、不使用灰度观察和不添加灰度新轴。

### 参数
- **env** (`Env`): 要应用预处理的环境。
- **noop_max** (`int`): No-op重置的最大no-ops动作数量。
- **frame_skip** (`int`): 新观察之间的帧数，影响代理体验游戏的频率。
- **screen_size** (`int`): 调整Atari帧的大小。
- **terminal_on_life_loss** (`bool`): 如果为True，当生命损失时`step()`返回`terminated=True`。
- **grayscale_obs** (`bool`): 如果为True，则返回灰度观察结果，否则返回RGB观察结果。
- **grayscale_newaxis** (`bool`): 如果为True且`grayscale_obs=True`，则给灰度观察添加一个通道轴使其变为三维。
- **scale_obs** (`bool`): 如果为True，则返回[0,1)范围内归一化的观察结果。

### 更改日志
- 在`gym v0.12.2`中添加。

该包装器通过提供标准化的预处理步骤，简化了Atari环境的使用，并有助于在不同研究和实验中保持一致性。

## 不常见包装器 Uncommon Wrappers

### class gymnasium.wrappers.Autoreset(env: Env)

`gymnasium.wrappers.Autoreset`是一个自动重置环境的包装器，当环境达到终止（`terminated=True`）或截断（`truncated=True`）状态时自动调用`Env.reset()`。这个包装器简化了环境重置的管理，特别是在进行长时间的训练或测试时非常有用，因为它自动处理环境的重置过程，无需手动干预。

### 返回格式
当调用`step`导致`Env.step()`返回`terminated=True`或`truncated=True`时，将自动调用`Env.reset()`，并且`self.step()`的返回格式如下：
- **new_obs**：调用`self.env.reset()`后的第一个观察值。
- **final_reward**：调用`self.env.step()`后的奖励，调用`self.env.reset()`之前。
- **final_terminated**：调用`self.env.reset()`之前的`terminated`值。
- **final_truncated**：调用`self.env.reset()`之前的`truncated`值。`final_terminated`和`final_truncated`不能同时为`False`。
- **info**：一个字典，包含从调用`self.env.reset()`返回的`info`字典中的所有键，以及额外的键“final_observation”和“final_info”，分别包含最后一次调用`self.env.step()`返回的观察值和`info`字典。

### 注意事项
使用这个包装器收集轨迹时，请注意，当`Env.step()`返回`terminated`或`truncated`时，`Env.step()`将返回调用`Env.reset()`后的新观察值，以及上一个剧集的最终奖励、终止和截断状态。如果你需要上一个剧集的最终状态，你需要通过`info`字典中的“final_observation”键检索它。使用这个包装器时，请确保你了解你正在做什么！

### 更改日志
- `v0.24.0`：最初作为`AutoResetWrapper`添加。
- `v1.0.0`：重命名为`Autoreset`，并且改变了自动重置的顺序，现在在环境终止或截断后的下一个步骤上重置。结果，“final_observation”和“final_info”被移除。

### 使用示例
```python
import gymnasium as gym

# 创建环境并应用Autoreset包装器
env = gym.make("CartPole-v1")
env = gym.wrappers.Autoreset(env)

# 运行环境并观察自动重置行为
obs = env.reset()
done = False
while not done:
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
    if terminated or truncated:
        print("Environment automatically reset.")
        break
```
这个示例展示了如何使用`Autoreset`包装器在环境达到终止或截断状态时自动重置环境，从而简化了环境管理过程。

### class gymnasium.wrappers.PassiveEnvChecker(env: Env[ObsType, ActType])

`gymnasium.wrappers.PassiveEnvChecker`是一个被动的包装器，用于环绕`step`、`reset`和`render`函数，检查它们是否遵循Gymnasium的API规范。这个包装器在使用`make`创建环境时会自动应用，并且可以通过`disable_env_checker`参数禁用。该包装器没有向量化环境的版本。

### 功能
- **API遵循性检查**：确保环境的`step`、`reset`和`render`函数遵循Gymnasium规定的API。
- **自动应用**：在使用`gym.make`创建环境时自动应用，无需手动添加。
- **可选禁用**：可以通过在`gym.make`时设置`disable_env_checker=True`来禁用这个包装器。

### 示例代码
```python
import gymnasium as gym

# 创建环境时自动应用PassiveEnvChecker
env = gym.make("CartPole-v1")
print(env)
# 输出：<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

# 创建环境时禁用PassiveEnvChecker
env = gym.make("CartPole-v1", disable_env_checker=True)
print(env)
# 输出：<TimeLimit<OrderEnforcing<CartPoleEnv<CartPole-v1>>>>
```

### 更改日志
- **v0.24.1**：最初添加，但存在多个问题。
- **v0.25.0**：修复了所有问题。
- **v0.29.0**：移除了对`Box`观察空间和动作空间的无限界限以及不规则界限形状的警告。

### 使用场景
- **开发新环境**：在开发新的Gym环境时使用，以确保其遵循Gymnasium的API规范。
- **环境调试**：在环境出现问题时使用，以帮助发现可能的API不一致问题。
- **提高代码质量**：自动检查环境的API遵循性，提高环境的代码质量和兼容性。

`PassiveEnvChecker`通过提供一个简单的方式来确保环境遵循Gymnasium的API规范，从而帮助开发者提高环境的质量和兼容性。通过可选的禁用功能，开发者还可以在不需要这些检查的情况下灵活使用环境。

### class gymnasium.wrappers.HumanRendering(env: Env[ObsType, ActType])

`gymnasium.wrappers.HumanRendering`包装器允许支持“rgb_array”渲染模式的环境进行类似人类的渲染。这个包装器特别适用于那些能够生成RGB图像但尚未实现将图像渲染到屏幕上代码的环境。如果你想在你的环境中使用这个包装器，记得在环境的元数据中指定`"render_fps"`。

包装后的环境的`render_mode`必须是`'rgb_array'`或`'rgb_array_list'`。

### 使用场景
- 当你的环境能够生成图像数据但没有实现图像渲染功能时，可以通过`HumanRendering`包装器将图像渲染到屏幕上。
- 在进行环境测试和调试时，如果想直观地看到环境状态变化，可以使用这个包装器。

### 示例
```python
import gymnasium as gym
from gymnasium.wrappers import HumanRendering

# 创建环境并设置渲染模式为"rgb_array"
env = gym.make("LunarLander-v2", render_mode="rgb_array")
wrapped = HumanRendering(env)

# 重置环境，开始将图像渲染到屏幕
obs, _ = wrapped.reset()
```
这个示例创建了一个`LunarLander-v2`环境，并通过`HumanRendering`包装器将环境渲染到屏幕上。这种方式对于可视化环境的状态非常有用，尤其是在开发和调试阶段。

### 注意事项
- 如果基础环境使用`render_mode="rgb_array_list"`，则其`render`方法将始终返回一个空列表。这意味着在使用`HumanRendering`包装器时，应避免将`render_mode`设置为`"rgb_array_list"`，以确保能够正常渲染图像到屏幕。

### 参数
- **env**：被包装的环境实例。

`HumanRendering`包装器没有向量化版本，因此它主要用于单个环境实例。使用这个包装器可以简化环境渲染到屏幕的流程，使环境开发者无需编写额外的渲染代码即可进行环境的可视化。

### class gymnasium.wrappers.OrderEnforcing(env: Env[ObsType, ActType], disable_render_order_enforcing: bool = False)

`gymnasium.wrappers.OrderEnforcing`是一个用于确保在调用`env.step()`或`env.render()`之前必须先调用`env.reset()`的包装器。如果没有先调用`reset()`而直接调用`step()`或`render()`，该包装器将产生错误。这个功能对于确保环境的使用顺序正确非常有用，可以避免因顺序错误导致的潜在bug。

### 参数
- **env** (`Env`): 要包装的环境。
- **disable_render_order_enforcing** (`bool`): 如果设置为`True`，则禁用对`render`函数调用顺序的强制检查。

### 使用场景
- **开发和调试**：在开发或调试环境时，确保环境的使用顺序正确，防止因为调用顺序错误而产生难以追踪的错误。
- **教学和演示**：在教学或演示环境使用时，确保展示的顺序符合预期，避免因操作顺序错误而导致的混乱。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import OrderEnforcing

# 创建环境并应用OrderEnforcing包装器
env = gym.make("CartPole-v1", render_mode="human")
env = OrderEnforcing(env)

# 在调用reset之前尝试step，将引发错误
try:
    env.step(0)
except gymnasium.error.ResetNeeded as e:
    print(e)  # 输出: Cannot call env.step() before calling env.reset()

# 在调用reset之前尝试render，将引发错误
try:
    env.render()
except gymnasium.error.ResetNeeded as e:
    print(e)  # 输出: Cannot call `env.render()` before calling `env.reset()`

# 正确的顺序：先reset，然后render和step
_ = env.reset()
env.render()
_ = env.step(0)
env.close()
```

使用`OrderEnforcing`包装器可以帮助开发者和用户避免因忘记调用`reset()`而直接进行`step()`或`render()`操作的常见错误，提高环境使用的健壮性。

### class gymnasium.wrappers.RenderCollection(env: Env[ObsType, ActType], pop_frames: bool = True, reset_clean: bool = True)

`gymnasium.wrappers.RenderCollection`是一个用于收集环境渲染帧的包装器，使`render`方法返回一个由渲染帧组成的列表。这个包装器特别适用于需要收集和分析渲染帧数据的场景，比如视频生成、可视化分析等。

### 参数
- **env** (`Env`): 被包装的环境。
- **pop_frames** (`bool`): 如果为`True`，在调用`render`方法后清空收集的帧列表。默认值为`True`。
- **reset_clean** (`bool`): 如果为`True`，在调用`reset`方法时清空收集的帧列表。默认值为`True`。

### 使用场景
- **帧收集**：在环境的一次或多次运行中收集渲染帧，用于生成视频或进行可视化分析。
- **数据分析**：收集的帧可以用于进一步的数据分析，比如帧间差异分析、特征提取等。

### 示例代码
#### 示例1：收集一系列操作后的所有帧并在`render`后清除
```python
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = gym.wrappers.RenderCollection(env)
_ = env.reset(seed=123)
for _ in range(5):
    _ = env.step(env.action_space.sample())

frames = env.render()
print(f"Number of frames collected: {len(frames)}")  # 输出：6

frames = env.render()
print(f"Number of frames after second render: {len(frames)}")  # 输出：0
```

#### 示例2：收集一系列操作后的所有帧并保留，不在`render`后清除
```python
env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = gym.wrappers.RenderCollection(env, pop_frames=False)
_ = env.reset(seed=123)
for _ in range(5):
    _ = env.step(env.action_space.sample())

frames = env.render()
print(f"Number of frames collected: {len(frames)}")  # 输出：6

frames = env.render()
print(f"Number of frames after second render: {len(frames)}")  # 输出：6
```

#### 示例3：在多次`reset`后收集所有帧，不在`render`或`reset`后清除
```python
env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = gym.wrappers.RenderCollection(env, pop_frames=False, reset_clean=False)
_ = env.reset(seed=123)
for _ in range(5):
    _ = env.step(env.action_space.sample())

_ = env.reset(seed=123)
for _ in range(5):
    _ = env.step(env.action_space.sample())

frames = env.render()
print(f"Total number of frames collected across resets: {len(frames)}")  # 输出：12
```


使用`RenderCollection`包装器可以灵活地管理和收集环境渲染帧，便于后续的视频生成、数据分析或可视化需求。通过调整`pop_frames`和`reset_clean`参数，可以根据需要保留或清除收集到的帧。

## 数据转换包装器 Data Conversion Wrappers

### class gymnasium.wrappers.JaxToNumpy(env: Env[ObsType, ActType])

`gymnasium.wrappers.JaxToNumpy`是一个包装器，用于将基于Jax的环境包装起来，使其可以通过NumPy数组进行交互。在这个包装器的帮助下，动作必须以NumPy数组的形式提供，观察结果将作为NumPy数组返回。这对于那些习惯于使用NumPy进行数据处理的用户来说非常有用，可以更容易地将基于Jax的环境集成到现有的基于NumPy的工作流程中。

### 注意事项
- **Jax到NumPy的转换**：并不保证可以完全无损地进行来回转换（即从Jax到NumPy再到Jax，反之亦然）。这是因为Jax不支持非数组值，因此像`numpy int_32(5)`这样的值在转换为Jax时可能会变成`DeviceArray([5], dtype=jnp.int23)`，这可能会影响到某些基于精确类型匹配的操作。

### 示例代码
```python
import gymnasium as gym

# 假设存在一个基于Jax的环境"JaxEnv-vx"（此环境为示例，可能并不存在）
env = gym.make("JaxEnv-vx")
env = gym.wrappers.JaxToNumpy(env)

# 重置环境，并检查观察值的类型
obs, _ = env.reset(seed=123)
print(type(obs))  # 输出：<class 'numpy.ndarray'>

# 采样一个动作，并执行一步动作，检查返回值的类型
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
print(type(obs))  # 输出：<class 'numpy.ndarray'>
print(type(reward))  # 输出：<class 'float'>
print(type(terminated))  # 输出：<class 'bool'>
print(type(truncated))  # 输出：<class 'bool'>
```



通过使用`JaxToNumpy`包装器，可以方便地将基于Jax的环境与基于NumPy的分析和处理工具链结合起来，无需担心数据格式转换问题。此外，它还提供了一个向量化版本`gymnasium.wrappers.vector.JaxToNumpy`，可以用于同时处理多个环境实例，进一步增强了其适用性和灵活性。

### class gymnasium.wrappers.JaxToTorch(env: gym.Env, device: Device | None = None)

`gymnasium.wrappers.JaxToTorch`是一个包装器，用于将基于Jax的环境包装起来，使其可以与PyTorch张量（Tensors）进行交互。通过这个包装器，动作需要以PyTorch张量的形式提供，观察结果也将作为PyTorch张量返回。这个包装器特别适用于在PyTorch项目中使用基于Jax的环境，使得数据处理和模型训练更加方便。

### 注意事项
- **渲染输出**：需要注意的是，对于渲染的输出，它将作为NumPy数组返回，而不是PyTorch张量。

### 示例代码
```python
import torch
import gymnasium as gym

# 假设存在一个基于Jax的环境"JaxEnv-vx"（此环境为示例，可能并不存在）
env = gym.make("JaxEnv-vx")
env = gym.wrappers.JaxToTorch(env)

# 重置环境，并检查观察值的类型
obs, _ = env.reset(seed=123)
print(type(obs))  # 输出：<class 'torch.Tensor'>

# 采样一个动作，并将其转换为PyTorch张量，执行一步动作，检查返回值的类型
action = torch.tensor(env.action_space.sample())
obs, reward, terminated, truncated, info = env.step(action)
print(type(obs))  # 输出：<class 'torch.Tensor'>
print(type(reward))  # 输出：<class 'float'>
print(type(terminated))  # 输出：<class 'bool'>
print(type(truncated))  # 输出：<class 'bool'>
```

### 参数
- **env** (`gym.Env`): 要包装的基于Jax的环境。
- **device** (`torch.device` 或 `None`): PyTorch张量应该移动到的设备。如果为`None`，则使用默认设备。


通过使用`JaxToTorch`包装器，可以将基于Jax的环境无缝地集成到PyTorch的工作流程中，从而使得在PyTorch环境下进行强化学习研究和开发变得更加简单和高效。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.JaxToTorch`，支持同时处理多个环境实例，进一步提高了处理效率。

### class gymnasium.wrappers.NumpyToTorch(env: gym.Env, device: Device | None = None)

`gymnasium.wrappers.NumpyToTorch`是一个包装器，用于将基于NumPy的环境包装起来，使其可以与PyTorch张量（Tensors）进行交互。通过这个包装器，动作必须以PyTorch张量的形式提供，观察结果也将作为PyTorch张量返回。这个包装器特别适合于那些希望在PyTorch项目中使用基于NumPy的环境的用户，它简化了数据转换过程，使得环境可以直接与PyTorch模型和算法协同工作。

### 注意事项
- **渲染输出**：对于渲染的输出，它将作为NumPy数组返回，而不是PyTorch张量。

### 示例代码
```python
import torch
import gymnasium as gym

# 创建基于NumPy的环境"CartPole-v1"
env = gym.make("CartPole-v1")
env = gym.wrappers.NumpyToTorch(env)

# 重置环境，并检查观察值的类型
obs, _ = env.reset(seed=123)
print(type(obs))  # 输出：<class 'torch.Tensor'>

# 采样一个动作，并将其转换为PyTorch张量，执行一步动作，检查返回值的类型
action = torch.tensor(env.action_space.sample(), dtype=torch.float)
obs, reward, terminated, truncated, info = env.step(action)
print(type(obs))  # 输出：<class 'torch.Tensor'>
print(type(reward))  # 输出：<class 'float'>
print(type(terminated))  # 输出：<class 'bool'>
print(type(truncated))  # 输出：<class 'bool'>
```

### 参数
- **env** (`gym.Env`): 要包装的基于NumPy的环境。
- **device** (`torch.device` 或 `None`): PyTorch张量应该移动到的设备。如果为`None`，则使用默认设备。


使用`NumpyToTorch`包装器可以方便地将基于NumPy的环境无缝集成到PyTorch的工作流程中，从而在PyTorch环境下进行强化学习研究和开发变得更加简单和高效。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.NumpyToTorch`，支持同时处理多个环境实例，进一步提高了处理效率。